In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

from cinnamon.drift import ModelDriftExplainer, AdversarialDriftExplainer

# Boston Housing Data

In [ ]:
boston = datasets.load_boston()

In [ ]:
boston_df = pd.DataFrame(boston.data, columns = boston.feature_names)

In [ ]:
boston_df.head()

In [ ]:
boston_df.shape

# Build XGBoost model

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(boston_df, boston.target, test_size=0.3, random_state=2021)

In [ ]:
model = XGBRegressor(n_estimators=1000,
                   booster="gbtree",
                   objective="reg:squarederror",
                   learning_rate=0.05,
                   max_depth=6,
                   seed=2021,
                   use_label_encoder=False)

In [ ]:
model.fit(X=X_train, y=y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=20, verbose=10)

# ModelDriftExplainer

The experiment is done with a random split so we should not detect a data drift

In [ ]:
drift_explainer = ModelDriftExplainer(model)

In [ ]:
drift_explainer.fit(X_train, X_valid, y_train, y_valid)

In [ ]:
# no apparent drift in distributions of predictions
drift_explainer.plot_prediction_drift()
drift_explainer.get_prediction_drift()

In [ ]:
# no apparent drift in distributions of target labels
drift_explainer.plot_target_drift()
drift_explainer.get_target_drift()

In [ ]:
# here we would detect a data drift but the difference in performance metrics
# is due to comparing train and valid dataset
drift_explainer.get_performance_metrics_drift()

In [ ]:
# plot drift values computed with the tree based approach
drift_explainer.plot_tree_based_drift_values()

In [ ]:
drift_explainer.get_tree_based_drift_values()

In [ ]:
# no apparent drift for LSTAT the first feature indicated by drift values
drift_explainer.plot_feature_drift('LSTAT')
drift_explainer.get_feature_drift('LSTAT')

# AdversarialDriftExplainer

In [ ]:
# logs corresponds training logs of adversarial classifiers
drift_explainer2 = AdversarialDriftExplainer().fit(X_train, X_valid, y_train, y_valid)

In [ ]:
# plot drift values computed with the adversarial approach
drift_explainer2.plot_adversarial_drift_values()
drift_explainer2.get_adversarial_drift_values()

In [ ]:
drift_explainer2.get_feature_drifts()